In [1]:
import pandas as pd
import os
import csv

In [2]:
def NormalizeStat(stat, min_, max_):
    return (stat - min_) / (max_ - min_)

In [85]:
df_full_game_log = pd.read_csv(os.path.join('..', "Resources", "full_game_logs.csv"))
df_players = pd.read_csv(os.path.join('..', "Resources", "players.csv"))

df_players.rename(inplace=True, 
    columns={
        'id':'player_id',
        'team':'team_name'
    })

df_full_game_log.rename(inplace=True, 
    columns={
        'year_id':'year',
        'gs':'started'
    })

df_full_game_log.game_location = ['Away' if l == '@' else 'Home' for l in df_full_game_log.game_location]
df_full_game_log.started = [True if s == '*' else False for s in df_full_game_log.started]

df_player_game_logs = df_full_game_log.merge(df_players.loc[:, ['player_id', 'position', 'team_name']], how='inner', on='player_id')

df_player_game_logs_2020 = df_player_game_logs.loc[(df_player_game_logs['year'] == 2020)]

df_rb_2020 = df_player_game_logs_2020.loc[(df_player_game_logs['position'] == 'RB')].dropna(how='all', axis=1).set_index('player_id')
df_qb_2020 = df_player_game_logs_2020.loc[(df_player_game_logs['position'] == 'QB')].dropna(how='all', axis=1).set_index('player_id')
df_wr_2020 = df_player_game_logs_2020.loc[(df_player_game_logs['position'] == 'WR')].dropna(how='all', axis=1).set_index('player_id')
df_te_2020 = df_player_game_logs_2020.loc[(df_player_game_logs['position'] == 'WR')].dropna(how='all', axis=1).set_index('player_id')

common_stats = ['week_num', 'team', 'opp', 'game_location', 'game_result', 'started']


In [95]:
df_teams_2020 = pd.DataFrame({
    'team': df_player_game_logs_2020['opp'].unique()
})

df_teams_2020.set_index('team', inplace=True)

dfgb_2020_opp_results = df_player_game_logs_2020.groupby('opp')

df_teams_2020['games_played'] = dfgb_2020_opp_results['game_date'].nunique()
df_teams_2020['rush_yds_allowed'] = dfgb_2020_opp_results['rush_yds'].sum()
df_teams_2020['avg_rush_yds_allowed_per_game'] = round(dfgb_2020_opp_results['rush_yds'].mean())

df_teams_2020

,games_played,rush_yds_allowed,avg_rush_yds_allowed_per_game
team,,,
TAM,6,385.0,8.0
LVR,5,634.0,14.0
GNB,5,579.0,14.0
DET,5,725.0,19.0
LAC,5,538.0,13.0
ATL,6,580.0,13.0
NWE,5,596.0,17.0
DAL,6,1040.0,20.0
MIA,6,745.0,15.0


# Running Back Analysis

In [55]:
rb_stats = ['rush_att', 'rush_yds', 'rush_yds_per_att', 'rush_td', 'fumbles', 'two_pt_md']
rb_stats = common_stats + rb_stats

df_rb_2020_gb = df_rb_2020.groupby(['player_id'])

df_rb_2020['starts'] = df_rb_2020_gb['started'].count()
df_rb_2020['total_rush_att'] = df_rb_2020_gb['rush_att'].sum()

df_rb_2020_reduced = df_rb_2020.loc[(df_rb_2020.starts >= 4) & (df_rb_2020.total_rush_att> 0), rb_stats].fillna(0)


df_rb_2020_reduced.groupby('player_id').describe()['rush_yds'].sort_values('mean', ascending=False)


,count,mean,std,min,25%,50%,75%,max
player_id,,,,,,,,
HenrDe00,5.0,117.600000,58.560225,57.0,84.00,116.0,119.0,212.0
CookDa01,5.0,97.800000,55.970528,50.0,63.00,65.0,130.0,181.0
SandMi01,5.0,86.800000,26.546186,46.0,80.00,95.0,95.0,118.0
EdwaCl00,6.0,84.166667,52.338959,38.0,46.00,64.0,119.5,161.0
ChubNi00,4.0,83.750000,38.439346,43.0,55.75,84.0,112.0,124.0
...,...,...,...,...,...,...,...,...
SherAn00,5.0,0.400000,0.894427,0.0,0.00,0.0,0.0,2.0
ProsC.00,5.0,0.200000,0.447214,0.0,0.00,0.0,0.0,1.0
RicaPa00,6.0,-0.166667,0.408248,-1.0,0.00,0.0,0.0,0.0
